In [ ]:
from ipynb.fs.defs.listings import *
from urllib.request import urlopen
from bs4 import BeautifulSoup
from random import randint
from time import sleep

import pickle5 as pickle
import pandas as pd
import requests
import os.path
import json
import re

In [ ]:
# Replace URL to get the list of advertisments
details_dict.clear()
details_dict = get_list("https://www.wg-gesucht.de/wg-zimmer-in-Passau.104.0.1.3.html")
print(details_dict)

In [ ]:
# Extract page and save a local copy of HTML
def extract(URL):
    print(URL)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    sleep(randint(2,10))
    
    #Download website locally (Replace the path with where the folder is located on your system)
    file_name = '/Users/ummerabab-/Desktop/wg-gesucht/Websites/' + URL.split("https://www.wg-gesucht.de/")[1].split('.html')[0] + '.txt'
    with open(file_name, "wb") as file:
        file.write(page.content)
            
    return soup

# Acquire useful information from HTML tags
def transform(soup):
    panel_body = soup.find('div', class_ = 'panel-body')
    rows = panel_body.find_all('div', class_ = 'row')
    
    wg_tag_str = ''
    
    for item in rows:
        #WG-Details
        for sub_item in item.find_all('div', class_ = 'col-sm-6'):
            if sub_item.h4.text.strip() == 'Die WG':
                ul_tag = sub_item.find('ul')
                for li in ul_tag.find_all('li'):
                    wg_tag_str = wg_tag_str + ';' + re.sub('\s+',' ',(li.text.strip().replace('\n','')))
                wg_tag.append(wg_tag_str)
 
            if sub_item.h4.text.strip() == 'Gesucht wird':
                ul_tag = sub_item.find('ul')
                for li in ul_tag.find_all('li'):
                    gesucht_wird.append(re.sub('\s+',' ',(li.text.strip().replace('\n',''))))
                    
        #Address
        if item.find('a', href = '#mapContainer'):
            adresse_str = re.sub('\s+',' ',(item.find('a', href = '#mapContainer').text.strip().replace('\n','')))
            adresse.append(adresse_str)
    
         #Person
#        if item.find('div', class_ = 'mb10'):
#            if item.find('img'):
#                person_img_src = item.find('img')['src']
#                 print(person_img_src['src'])


    
    #Description
    description_tag = panel_body.find('div', id = 'ad_description_text')
    #print(description_tag)

#     zimmer = ''
#     lage = ''
#     wg_leben = ''
#     sonstiges = ''
        
    if description_tag.find('div', id ='freitext_0'):
            zimmer.append(description_tag.find('p', id ='freitext_0_content').text.strip().replace('\n','').replace('\r',''))
    else:
            zimmer.append('')
    if description_tag.find('div', id ='freitext_1'):
            lage.append(description_tag.find('p', id ='freitext_1_content').text.strip().replace('\n','').replace('\r',''))
    else:
            lage.append('')
    if description_tag.find('div', id ='freitext_2'):
            wg_leben.append(description_tag.find('p', id ='freitext_2_content').text.strip().replace('\n','').replace('\r',''))
    else:
            wg_leben.append('')
    if description_tag.find('div', id ='freitext_3'):
            sonstiges.append(description_tag.find('p', id ='freitext_3_content').text.strip().replace('\n','').replace('\r',''))
    else:
            sonstiges.append('')
            

    # Set values in details_dict
    details_dict['Die_WG'] = wg_tag
    details_dict['Gesucht_wird'] = gesucht_wird
    details_dict['Adresse']= adresse
    details_dict['Zimmer'] = zimmer
    details_dict['Lage'] = lage
    details_dict['WG-Leben'] = wg_leben
    details_dict['Sonstiges'] = sonstiges

    return details_dict

In [ ]:
url_array = details_dict['URL']
print(url_array)

In [ ]:
wg_tag = []
gesucht_wird = []
adresse = []
zimmer = []
lage = []
wg_leben = []
sonstiges = []

# Loop through 20 advertisments per page
for url in url_array:
    
    #Specify the URL to extract from
    page = extract(url)
    # Call functions to Extract HTML from URL and Transform it.
    details_dict = transform(page)

print(details_dict)

In [ ]:
print(len(details_dict['Name']))
print(len(details_dict['URL']))
print(len(details_dict['Title']))
print(len(details_dict['Die_WG']))
print(len(details_dict['Gesucht_wird']))
print(len(details_dict['Adresse']))
print(len(details_dict['Zimmer']))
print(len(details_dict['WG-Leben']))
print(len(details_dict['Sonstiges']))

In [ ]:
# Convert dictionary to data_frame
#data_frame = pd.DataFrame([details_dict])
data_frame = pd.DataFrame.from_dict(details_dict)
print (data_frame)

# Save to file with pickle in csv format
csv_path = '/Users/ummerabab-/Desktop/wg-gesucht/Code/dataset.csv'
if(os.path.isfile(csv_path)):
    data_frame.to_csv(csv_path, mode='a', encoding='utf-8', index= False, header=False) #
else:
    data_frame.to_csv(csv_path, encoding='utf-8', index= False)
#data_frame.to_csv(r'/Users/ummerabab-/Desktop/wg-gesucht/Code/dataset.csv', index = False)